<a href="https://colab.research.google.com/github/tujmon/trabalho_bio/blob/master/P5_Busca_de_Motivos_em_Sequ%C3%AAncias_Conservadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import math

In [ ]:
def strip_lines(fasta):
  seq = []
  for line in fasta:
    if line[0] != ">":
        line = line.strip()
        seq.append(line)
  return seq

def populate_array(array, length):
  for position in range(1, len(array)):
    array[position] = round(array[position]/length, 2)
  return array

def matrix(name_fastaFile):

  fastaFile = open(name_fastaFile, 'r')
  sequences = []
  sequences = strip_lines(fastaFile)

  A = ['A']
  C = ['C']
  U = ['U']
  G = ['G']

  for element in [A, C, U, G]:
    while len(element) < (len(sequences[0])+1):
      element.append(0)

  total_a = 0
  total_u = 0
  total_c = 0
  total_g = 0
  total_bases = 0

  for element in sequences:
    for i in range(0, len(sequences[0])):
      if element[i] == 'A':
        A[i+1] += 1
        total_a += 1
      elif element[i] == 'C':
        C[i+1] += 1
        total_c += 1
      elif element[i] == 'U':
        U[i+1] += 1
        total_u += 1
      elif element[i] == 'G':
        G[i+1] += 1
        total_g += 1

      total_bases += 1
  
  A = populate_array(A, len(sequences))
  C = populate_array(C, len(sequences))
  U = populate_array(U, len(sequences))
  G = populate_array(G, len(sequences))

  A.append(round(total_a/total_bases, 2))
  C.append(round(total_c/total_bases, 2))
  U.append(round(total_u/total_bases, 2))
  G.append(round(total_g/total_bases, 2))

  numbers = []
  for i in range(1, len(sequences[0]) + 1):
    numbers.append(i)
  matrix = [ ['Pos.', *numbers, "Overall freq."], A, C, U, G ]
  return matrix
array_c = matrix("motif_c.fasta")
array_d = matrix("motif_d.fasta")

In [ ]:
def frequency_matrix(matriz):
  result = np.array(matriz)
  result_pandas = pd.DataFrame(data = result[1:, 1:], index = result[1:, 0], columns = result[0, 1:])
  display(result_pandas)
print("-------------------------------- motif_C --------------------------------")
frequency_matrix(array_c)
print("-------------------------------- motif_D --------------------------------")
frequency_matrix(array_d)

-------------------------------- motif_C --------------------------------


,1,2,3,4,5,6,7,8,Overall freq.
A,0.32,0.0,0.0,1.0,0.0,0.0,1.0,0.11,0.3
C,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0
U,0.01,1.0,0.0,0.0,1.0,0.0,0.0,0.53,0.32
G,0.66,0.0,1.0,0.0,0.0,1.0,0.0,0.36,0.38


-------------------------------- motif_D --------------------------------


,1,2,3,4,5,6,Overall freq.
A,0.0,0.0,0.0,0.0,0.99,0.0,0.16
C,0.0,1.0,0.0,0.0,0.0,0.01,0.17
U,0.99,0.0,1.0,0.0,0.01,0.26,0.38
G,0.01,0.0,0.0,1.0,0.0,0.72,0.29


In [ ]:
def normalized_matrix_sequence(matrix):
  for i in range(1, 5):
    for j in range(1, len(matrix[i])-1):
      if (matrix[i][-1] == 0):
        matrix[i][j] = 0
      else:
        matrix[i][j] /= matrix[i][-1]
      matrix[i][j] = round(matrix[i][j], 2)
      if (matrix[i][j] > 0) :
       matrix[i][j] = round(math.log2(matrix[i][j]), 2)
      else:
        avrg = 0
        min = 100
        for position in range (1, 5):
          if (matrix[i][position] > 0 and matrix[i][position] < min):
            min = matrix[i][position]
          avrg += matrix[i][position]
        matrix[i][j] = round( (min - (avrg/8)), 2)
  
  result = np.array(matrix)

  result_pandas = pd.DataFrame(data = result[1:, 1:], index = result[1:, 0], columns = result[0, 1:])

  display(result_pandas)
print("-------------------------------- motif_C --------------------------------")
normalized_matrix_sequence(array_c)
print("-------------------------------- motif_D --------------------------------")
normalized_matrix_sequence(array_d)

-------------------------------- motif_C --------------------------------


,1,2,3,4,5,6,7,8,Overall freq.
A,0.1,-0.04,-0.03,1.74,-0.12,-0.12,1.74,-1.43,0.3
C,100.0,87.5,64.06,32.62,-2.9,-2.9,-2.9,-2.9,0.0
U,-5.06,1.64,2.07,1.81,1.64,1.58,1.58,0.73,0.32
G,0.8,0.58,1.4,0.23,-0.15,1.4,-0.15,-0.07,0.38


-------------------------------- motif_D --------------------------------


,1,2,3,4,5,6,Overall freq.
A,100.0,87.5,64.06,32.62,2.63,-2.9,0.16
C,0.88,2.56,0.45,-0.04,-0.03,-4.06,0.17
U,1.38,0.7,1.4,0.26,-5.06,-0.56,0.38
G,-5.06,1.51,1.32,1.79,1.38,1.31,0.29
